In [1]:
# Step 1: Import the necessary libraries
import pandas as pd
import statsmodels.api as sm
from pandas_datareader import data as pdr

In [2]:
# Step 2: Load the data
file_path = './data/ftw_spx_2024.csv'
strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)

/tmp/ipykernel_95654/58061667.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)


In [3]:
# Step 3: Separate SPX and strategies
universe_prices = strategies_prices['SPX']
risk_premia_ftw_prices = strategies_prices.drop(columns=['SPX'])

# Convert the strategies to returns
strategies = risk_premia_ftw_prices.pct_change().dropna()
universe = universe_prices.pct_change().dropna()

In [4]:
# Step 4: Retrieve the Fama-French 4-factor data
ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]

# Combine the Fama-French factors and the momentum factor into a single DataFrame
ff_factors = ff_factors.join(momentum_factor['Mom   '])

# Convert the factors to daily returns as percentages
ff_factors = ff_factors / 100

/tmp/ipykernel_95654/3849217727.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
/tmp/ipykernel_95654/3849217727.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]


In [5]:
# Step 5: Run the Fama-French 4-factor regression for each strategy
model_results = {}
for strategy_name in strategies.columns:
    # Merge strategy returns with the Fama-French factors
    data = pd.concat([strategies[strategy_name], ff_factors], axis=1, join='inner')
    
    # Prepare the data for regression
    X = data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
    y = data[strategy_name] - data['RF']
    
    # Add a constant to the independent variables
    X = sm.add_constant(X)
    
    # Run the regression
    model = sm.OLS(y, X).fit()
    
    # Store the model results
    model_results[strategy_name] = model

In [6]:
# Extract the coefficients and p-values for each strategy into 2 distinctive tables
coefficients = pd.DataFrame({strategy: model_results[strategy].params for strategy in model_results})
p_values = pd.DataFrame({strategy: model_results[strategy].pvalues for strategy in model_results})

# Round the values to 3 decimal places
coefficients = coefficients.round(3)
p_values = p_values.round(3)

# Step 6: Display the results
print('Coefficients:')
print(coefficients)
print('\nP-Values:')
print(p_values)

Coefficients:
        Quality  LowVol  Value  Momentum
const     0.000   0.001  0.001     0.000
Mkt-RF    0.828   0.445  0.837     0.966
SMB       0.008  -0.142 -0.004     0.114
HML       0.026  -0.076  0.565     0.103
Mom      -0.142  -0.520 -0.440     0.284

P-Values:
        Quality  LowVol  Value  Momentum
const     0.860   0.022  0.001     0.283
Mkt-RF    0.000   0.000  0.000     0.000
SMB       0.834   0.013  0.931     0.011
HML       0.645   0.328  0.000     0.100
Mom       0.004   0.000  0.000     0.000


In [7]:
# Add the universe results to these two tables
universe_data = pd.concat([universe, ff_factors], axis=1, join='inner')
X = universe_data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
y = universe_data['SPX'] - universe_data['RF']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
universe_coefficients = model.params.round(3)
universe_p_values = model.pvalues.round(3)

print('\nUniverse Coefficients:')
print(universe_coefficients)
print('\nUniverse P-Values:')
print(universe_p_values)



Universe Coefficients:
const    -0.000
Mkt-RF    0.992
SMB      -0.095
HML      -0.034
Mom      -0.026
dtype: float64

Universe P-Values:
const     0.574
Mkt-RF    0.000
SMB       0.000
HML       0.000
Mom       0.001
dtype: float64


In [8]:
# Merge universe resuts with the strategy results without append method 
coefficients = pd.concat([coefficients, universe_coefficients], axis=1)
p_values = pd.concat([p_values, universe_p_values], axis=1)
# Add "S&P500" as its name
coefficients.columns = coefficients.columns.tolist()[:-1] + ['S&P500']
p_values.columns = p_values.columns.tolist()[:-1] + ['S&P500']


# Display the updated tables
print('\nUpdated Coefficients:')
print(coefficients)
print('\nUpdated P-Values:')
print(p_values)


Updated Coefficients:
        Quality  LowVol  Value  Momentum  S&P500
const     0.000   0.001  0.001     0.000  -0.000
Mkt-RF    0.828   0.445  0.837     0.966   0.992
SMB       0.008  -0.142 -0.004     0.114  -0.095
HML       0.026  -0.076  0.565     0.103  -0.034
Mom      -0.142  -0.520 -0.440     0.284  -0.026

Updated P-Values:
        Quality  LowVol  Value  Momentum  S&P500
const     0.860   0.022  0.001     0.283   0.574
Mkt-RF    0.000   0.000  0.000     0.000   0.000
SMB       0.834   0.013  0.931     0.011   0.000
HML       0.645   0.328  0.000     0.100   0.000
Mom       0.004   0.000  0.000     0.000   0.001
